In [1]:
import qiskit as qk
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.circuit.random import random_circuit
from qiskit.providers.ibmq import least_busy
from context import tools as t
from functools import partial

In [2]:
class backend:
    '''
    This class serves as a namespace for the ibmq quantum computer to be used
    in a run of code. Calling backend.ibm() will check if ibm_backend is already
    chosen, if not it chooses it and returns it.
    '''
    blacklist = ['ibmq_bogota']
    job_manager = IBMQJobManager()
    sim_backend = qk.Aer.get_backend('qasm_simulator')
    ibm_backend = None
    _provider = None

    def __init__(self):
        pass
    
    @classmethod
    def provider(cls):
        if cls._provider == None:
            cls._provider = qk.IBMQ.load_account()
        return cls._provider

    @classmethod
    def sim(cls):
        return cls.sim_backend

    @classmethod
    def ibm(cls):
        if cls.ibm_backend == None:
            provider = cls.provider()
#             provider = qk.IBMQ.get_provider(hub='ibm-q')
            filters = lambda x:(
                x.status().operational and
                not x.configuration().simulator and
                x.configuration().n_qubits > 1 and
                x.configuration().backend_name not in cls.blacklist
            )
            adequate_backends = provider.backends(filters=filters)
            cls.ibm_backend = least_busy(adequate_backends)
        return cls.ibm_backend

In [3]:
qc = random_circuit(num_qubits=5, depth=4, measure=True)
type(qc)

qiskit.circuit.quantumcircuit.QuantumCircuit

In [4]:
print(backend.ibm_backend)

None


In [45]:
def send_jobs_ibm(circs, name):
    b = backend.ibm()
    
    if not isinstance(circs, list):
        circs = [circs]
        
    circs_transpiled = qk.transpile(circs, backend=b)
    job_set = backend.job_manager.run(circs_transpiled, backend=b, name=name)
    return job_set.job_set_id()
job_id = send_jobs_ibm([qc, qc], 'test2')

In [31]:
def get_counts(job_set_id):
    job_id = 'f4db1967dca64005bb248898049743ef-16389803640987256'
    job_set = backend.job_manager.retrieve_job_set(job_set_id=job_id, provider=backend.provider())
    counts = [job_set.results().get_counts(i) for i in range(len(job_set.jobs()))]
    return 
    
results = get_results('f4db1967dca64005bb248898049743ef-16389803640987256')

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [27]:
dir()

TypeError: 'method' object is not subscriptable

In [44]:
job_id = 'f4db1967dca64005bb248898049743ef-16389803640987256'
job_set = backend.job_manager.retrieve_job_set(job_set_id=job_id, provider=backend.provider())
counts = [job_set.results().get_counts(i) for i in range(len(job_set.jobs()))]
print(counts)

[{'00000': 163, '00001': 60, '10000': 33, '10001': 17, '10010': 20, '10011': 18, '10100': 25, '10101': 13, '10110': 22, '10111': 16, '11000': 21, '11001': 17, '11010': 13, '11011': 11, '11100': 11, '11101': 9, '11110': 22, '11111': 17, '00010': 46, '00011': 26, '00100': 61, '00101': 13, '00110': 24, '00111': 19, '01000': 89, '01001': 30, '01010': 67, '01011': 31, '01100': 37, '01101': 25, '01110': 25, '01111': 23}]


In [46]:
dir(job_set)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_backend',
 '_error_msg',
 '_id',
 '_id_long',
 '_id_prefix',
 '_id_suffix',
 '_job_submit_lock',
 '_managed_jobs',
 '_managed_results',
 '_name',
 '_parse_job_name',
 '_tags',
 'cancel',
 'error_messages',
 'job',
 'job_set_id',
 'jobs',
 'managed_jobs',
 'name',
 'qobjs',
 'report',
 'results',
 'retrieve_jobs',
 'run',
 'statuses',
 'tags',
 'update_name',
 'update_tags']

In [50]:
# job_id = 'f4db1967dca64005bb248898049743ef-16389803640987256'
job_id = '56a2f61126ef4dfd8ce9358df0771539-16389933487636719'
job_set = backend.job_manager.retrieve_job_set(job_id, backend.provider())
print(job_set.statuses())

[<JobStatus.QUEUED: 'job is queued'>]
